In [5]:
#streaming data and stopping during specific time of day 
from multiprocessing import Process, Manager
import time
import pyEX 

# Your foo function
def foo(tmp):
    c = pyEx.Client(api_token='pk_6353e33d5d1244c5b48150a9e1e93239')
    c.deepSSE('iex','book',on_data=tmp.append)
    time.sleep(1)
    #return tmp
if __name__ == '__main__':
    with Manager() as manager:
        # Start foo as a process
        l = manager.list(range(10))
        p = Process(target=foo, name="Foo", args=(l))
        p.start()

        # Wait 10 seconds for foo
        time.sleep(10)

        # Terminate foo
        #p.terminate()

        # Cleanup
        p.join()
        print('run')

Process Foo:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
TypeError: foo() takes 1 positional argument but 10 were given


run


In [1]:
import sys
import qlib 
from qlib.data import D
import qlib
import pandas as pd
from qlib.config import REG_CN
from qlib.contrib.model.gbdt import LGBModel
from qlib.contrib.data.handler import Alpha158
from qlib.contrib.strategy.strategy import TopkDropoutStrategy
from qlib.contrib.evaluate import (
    backtest as normal_backtest,
    risk_analysis,
)
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict
#Will update processor as we start to process other types of data ie level 2 
#process_level1=processor.Processor()
#initialize which folder you would like to work in 
qlib.init(provider_uri='~/.qlib/qlib_data/L2')
instruments=D.instruments(market='all')
calendar=D.calendar(start_time='2021-02-25 01:47:13', end_time='2021-02-25 02:51:09', freq='1s')#[:2]
fields = ['$price','$rsi','$size','$type']
features=D.features(instruments,fields,start_time='2021-02-25 01:47:13', end_time='2021-02-25 02:51:09',freq='1s')

[88960:MainThread](2021-03-03 14:49:49,921) INFO - qlib.Initialization - [config.py:277] - default_conf: client.
[88960:MainThread](2021-03-03 14:49:49,923) WARNING - qlib.Initialization - [config.py:293] - redis connection failed(host=127.0.0.1 port=6379), cache will not be used!
[88960:MainThread](2021-03-03 14:49:49,925) INFO - qlib.Initialization - [__init__.py:46] - qlib successfully initialized based on client settings.
[88960:MainThread](2021-03-03 14:49:49,926) INFO - qlib.Initialization - [__init__.py:47] - data_path=/Users/Alexis/.qlib/qlib_data/L2


In [15]:
from qlib.data.dataset.loader import QlibDataLoader
fields = ['$price'] # MACD
names = ['$price']
labels = ['$rsi'] # label
label_names = ['$rsi']
data_loader_config = {
"feature": (fields, names),
"label": (labels, label_names)
}
data_loader = QlibDataLoader(config=data_loader_config)
df = data_loader.load(instruments=instruments,start_time='2021-02-25 01:47:13', end_time='2021-02-25 02:51:09', freq='1s')


In [16]:
df

,,feature,label
,,$price,$rsi
datetime,instrument,,
2021-02-25 01:47:13,L2,197.460007,55.525108
2021-02-25 01:48:00,L2,197.509995,55.527699
2021-02-25 01:48:38,L2,197.410004,55.520733
2021-02-25 01:48:41,L2,197.410004,55.520733
2021-02-25 01:51:04,L2,197.410004,55.520733
...,...,...,...
2021-02-25 02:50:47,L2,197.270004,55.503883
2021-02-25 02:50:53,L2,196.970001,52.026424


In [13]:
import xgboost as xgb
from qlib.contrib.model import xgboost
model=xgboost.XGBModel()

In [14]:
model.fit(df)

AttributeError: 'DataFrame' object has no attribute 'prepare'

In [27]:
#example works with level 1 IEX data 
###################################
# train model
###################################
data_handler_config = {
    "start_time": "2021-02-25 01:47:13",
    "end_time": "2021-02-25 02:51:09",
    "fit_start_time": "2021-02-25 01:47:13",
    "fit_end_time": "2021-02-25 02:51:09",
    "instruments": instruments,
}

task = {
    "model": {
        "class": "LGBModel",
        "module_path": "qlib.contrib.model.gbdt",
        "kwargs": {
            "loss": "mse",
            "colsample_bytree": 0.8879,
            "learning_rate": 0.0421,
            "subsample": 0.8789,
            "lambda_l1": 205.6999,
            "lambda_l2": 580.9768,
            "max_depth": 8,
            "num_leaves": 210,
            "num_threads": 20,
        },
    },
    "dataset": {
        "class": "DatasetH",
        "module_path": "qlib.data.dataset",
        "kwargs": {
            "handler": {
                "class": "Alpha158",
                "module_path": "qlib.contrib.data.handler",
                "kwargs": data_handler_config,
            },
            "segments": {
                "train": ("2021-02-25 01:47:13", "2021-02-25 02:51:09"),
                "valid": ("2021-02-25 01:47:13", "2021-02-25 02:51:09"),
                "test": ("2021-02-25 01:47:13", "2021-02-25 02:51:09"),
            },
        },
    },
}

# model initiaiton
model = init_instance_by_config(task["model"])
dataset = init_instance_by_config(task["dataset"])

# start exp to train model
with R.start(experiment_name="train_model"):
    R.log_params(**flatten_dict(task))
    model.fit(dataset)
    R.save_objects(trained_model=model)
    rid = R.get_recorder().id

TypeError: Can only swap levels on a hierarchical axis.